# k分割交差検証を使ったモデルの性能評価

In [26]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
# ジュピターノートブック上でグラフを表示させるための処理
%matplotlib inline

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

## Breast Cancer Wisconsinデータセットの読み込み

このデータセットには、悪性腫瘍細胞と良性腫瘍細胞の569のサンプルが含まれている。

このデータセットの最初の2つの列には、サンプルの一意なIDとそれに対応する診断結果が含まれている。診断結果の`M`は悪性(malignant)を示し、`B`は良性(benign)を示す。

3列目から32列目には、細胞核のデジタル画像から算出された30個の実数値の特徴量が含まれている。

これらは腫瘍が良性か悪性かを予測するモデルの構築に利用できる。

In [3]:
# データセットの読み込み
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.data', header=None)
# 30個の特徴量をnumpy配列のオブジェクトXに割り当てる。
# LabelEncoderを使って、元のクラスラベルの文字列表現(MおよびB)を整数に変換する。
from sklearn.preprocessing import LabelEncoder
X = df.loc[:, 2:].values
y = df.loc[:, 1].values
le = LabelEncoder()
# 配列yに格納されたクラスラベル（診断結果）エンコードした後は、悪性腫瘍はクラス1、良性腫瘍はクラス2で表される。
y = le.fit_transform(y)
print(y[:5])
print(le.fit_transform(['M', 'B']))

# エンコード後のデータセットをトレーニングデータセット（データの80%）とテストデータセット（データの20%）に分割する。
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1)

[1 1 1 1 1]
[1 0]


## パイプラインで変換器と推定器を結合する

トレーニングデータセットとテストデータセットの学習と変換を別々に行う代わりに、StandardScaler, 主成分分析(PCA), LogisticRegressionの3つのオブジェクトをパイプラインで結合できる。

主成分分析ではデータを最初の30次元から2次元の部分空間に圧縮する。

In [13]:
# 連結する処理としてスケーリング、主成分分析、ロジスティック回帰を指定
pipe_lr = Pipeline([('scl', StandardScaler()), ('pca', PCA(n_components=2)), ('clf', LogisticRegression(random_state=1))])
pipe_lr.fit(X_train, y_train)

print('Test Accuracy: %.3f' % pipe_lr.score(X_test, y_test))

Test Accuracy: 0.947


/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


## ホールドアウト法

ホールドアウト法は、機械学習のモデルの汎化性能を評価するために従来より使用されている一般的なアプローチである。

一般に機械学習を応用するには、未知のデータに対する予測性能をさらに向上させるために、さまざまなパラメータ設定のチューニングや比較を行うことが重要である。このプロセスを`モデル選択`と呼ぶ。

「モデル選択」という用語は、チューニングパラメータの「最適」な値を選択する分類問題を指す。チューニングパラメータは`ハイパーパラメータ`とも呼ばれる。

ただし、モデル選択において同じテストデータセットを繰り返し使用した場合、それはトレーニングデータセットの一部となる。このため、モデルが過学習に陥る可能性が高くなる。

モデル選択にホールドアウト法を使用する場合、より効果的な方法は、データをトレーニングデータセット、検証データセット、テストデータセットの3つに分割することである。
 - トレーニングデータセットはさまざまなモデルの学習に使用される。
 - 検証データセットでの性能は、モデル選択に使用される。

このようにトレーニングステップとモデル選択ステップで未知のテストデータセットを使用することにより、新しいデータを汎化する能力についてモデルを評価するときにバイアスが低くすることができる。
 
 ホールドアウト法の問題点として、元のトレーニングデータセットをトレーニングサブセットと検証サブセットにどのように分割するかによって、性能評価に影響がおよぶことがある。

## k分割交差検証

k分割交差検証では、非復元抽出を用いて、トレーニングデータセットをランダムにk個に分割する。そのうちのk-1個をモデルのトレーニングに使用し、1個をテストに使用する。この手順をk回繰り返すことで、k個のモデルと性能評価を取得する。
- つまり、トレーニングデータセットのk個のサブセットに対してホールドアウト法をk回繰り返す。

### 層化k分割交差検証イテレータ

層化交差検証では、各サブセットでのクラスの比率が維持される。この場合の比率は、トレーニングデータセット全体でのクラスの比率を表している。

In [25]:
# 分割元データ、分割数、乱数生成器の状態を指定し、層化k分割交差検証イテレータを表すStratifiedKFoldクラスのインスタンス化
kfold = StratifiedKFold(n_splits=10, random_state=1)

scores = []
# イテレータのインデックスと要素をループ処理: (上から順に)
#        データをモデルに適合
#        テストデータの正解率を算出
#        リストに正解率を追加
#        分割の番号、0以上の要素数、正解率を出力
for k, (train, test) in enumerate(kfold.split(X=X_train, y=y_train)):
    #print(train)
    pipe_lr.fit(X_train[train], y_train[train])
    score = pipe_lr.score(X_train[test], y_train[test])
    scores.append(score)
    print('Fold: %s, Class dist.: %s, Acc: %.3f' % (k+1, np.bincount(y_train[train]), score))

Fold: 1, Class dist.: [256 153], Acc: 0.891
Fold: 2, Class dist.: [256 153], Acc: 0.978
Fold: 3, Class dist.: [256 153], Acc: 0.978
Fold: 4, Class dist.: [256 153], Acc: 0.913
Fold: 5, Class dist.: [256 153], Acc: 0.935
Fold: 6, Class dist.: [257 153], Acc: 0.978
Fold: 7, Class dist.: [257 153], Acc: 0.933
Fold: 8, Class dist.: [257 153], Acc: 0.956
Fold: 9, Class dist.: [257 153], Acc: 0.978
Fold: 10, Class dist.: [257 153], Acc: 0.956


/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence t

In [23]:
# 正解率の平均と標準偏差を出力
print('CV accuracy: %.3f +/- %.3f' % (np.mean(scores), np.std(scores)))

CV accuracy: 0.950 +/- 0.029


⬆︎

まず、sklearn.model_selectionモジュールが提供するStratifiedKFoldイテレータオブジェクトを分割の個数をn_splitsパラメータを使って指定している。

StratifiedKFoldイテレータオブジェクトであるkfoldイテレータは、k個のサブセットをforループで処理するために使用する。trainによって返されたインデックス（`全インデックスの部分集合のインデックス`）を使用して、`X_trainの部分集合`と`y_trainの部分集合`を取得して、ロジスティック回帰パイプラインを学習している。

このpipe_lrパイプラインを使用することで、各イテレーションでサンプルが正しくスケーリング（標準化など）されるようにしている。

続いてtest(分割したトレーニングデータの内、検証データとして使用されるもの)を使ってモデルの正解率を計算している。そして、正解率の平均と標準偏差を計算するために、これをscoresリストにまとめている。

### scikit-learnによるk分割交差検証の性能指標を算出

In [27]:
# 交差検証のcross_val_score関数でモデルの正解率を算出
# 推定器estimator、トレーニングデータX、予測値y、分割数cv、CPU数n_jobsを指定
scores = cross_val_score(estimator=pipe_lr, X=X_train, y=y_train, cv=10, n_jobs=1)
print('CV accuracy scores: %s' % scores)
print('===')
print('CV accuracy: %.3f +/- %.3f' % (np.mean(scores), np.std(scores)))

CV accuracy scores: [0.89130435 0.97826087 0.97826087 0.91304348 0.93478261 0.97777778
 0.93333333 0.95555556 0.97777778 0.95555556]
===
CV accuracy: 0.950 +/- 0.029


/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence t

⬆︎

cross_val_score関数のアプローチのきわめて便利な特徴の1つとして、サブセットごとの評価の計算処理を`マシン上の複数のCPUに分散`させることができる。

今回の例のように、n_jobs引数を1に設定した場合、性能の評価に使用されるCPUは1つだけである。n_jobs=-1に設定した場合は、マシン上で利用可能なすべてのCPUを使って交差検証を並列実行できる。